In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df_train = pd.read_csv("train_18_10000.csv")
df_val = pd.read_csv("val_18_10000.csv")
df_test = pd.read_csv("test_18_10000.csv")

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf

In [ ]:
from tensorflow import keras

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
df_train_1d=np.array(df_train.drop("label", axis=1)).reshape(df_train.shape[0],df_train.shape[1]-1,1)
df_test_1d=np.array(df_test.drop("label", axis=1)).reshape(df_test.shape[0],df_test.shape[1]-1,1)
df_val_1d=np.array(df_val.drop("label", axis=1)).reshape(df_val.shape[0],df_val.shape[1]-1,1)

In [ ]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df_train['label_encoded'] = label_encoder.fit_transform(df_train['label'])
df_val['label_encoded'] = label_encoder.transform(df_val['label'])
df_test['label_encoded'] = label_encoder.transform(df_test['label'])

# One-hot encode the integer labels
y_train = to_categorical(df_train['label_encoded'])
y_val = to_categorical(df_val['label_encoded'])
y_test = to_categorical(df_test['label_encoded'])

# Verify the transformation
print("Original labels:", df_train['label'].unique())
print("Encoded labels:", df_train['label_encoded'].unique())
print("One-hot encoded shape:", y_train.shape)

num_classes = len(df_train['label'].unique())

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential

model = Sequential()

# Add model layers
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(df_train_1d.shape[1], 1)))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import time
start_time = time.time()
model.fit(df_train_1d, y_train, validation_data=(df_val_1d, y_val), epochs=10)
cnn_training_time = time.time() - start_time
print(f"CNN training time: {cnn_training_time:.2f} seconds")

In [ ]:
model.evaluate(df_test_1d, y_test)

591/591 [==============================] - 2s 2ms/step - loss: 0.1047 - accuracy: 0.8653


[0.10469944775104523, 0.8652613759040833]